In [ ]:
import os
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores.faiss import FAISS
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.prompts import PromptTemplate
from langchain.schema import StrOutputParser
from langchain_community.document_loaders import PyPDFDirectoryLoader

# OpenAI API 키 설정 (실제 사용 시에는 안전한 방법으로 API 키를 관리해야 함)
os.environ["OPENAI_API_KEY"] = "OPENAI_API_KEY"

In [2]:
# ======================
# [RAG 사용 pdf 파일정보]
# - 조간보도자료.pdf: 금융위원회에서 발간한 서비스 전세 대출 서비스에 대한 보도자료
# ======================
# 1. 폴더 내의 모든 pdf 로드하여 QA(Question-Answering) 챗봇으로 변경하는 코드

loader = PyPDFDirectoryLoader("../data", glob="*.pdf")
docs = loader.load()

print(f"문서의 수: {len(docs)}\n")
print("[메타데이터]\n")
print(docs[0].metadata)
print("========= [앞부분] 미리보기 =========")
print(docs[0].page_content[:300])
print("\n\n[메타데이터]\n")
print(docs[3].metadata)
print("========= [앞부분] 미리보기 =========")
print(docs[3].page_content[:300])

문서의 수: 10

[메타데이터]

{'producer': 'Call PDF v 2.4', 'creator': 'Call PDF', 'creationdate': '', 'title': '', 'author': '', 'subject': '', 'source': '..\\data\\조간보도자료.pdf', 'total_pages': 10, 'page': 0, 'page_label': '1'}
========= [앞부분] 미리보기 =========
- 1 -
금융위원회보도자료
보도시점2024. 1.31 (수) 조간배포2024. 1. 30.(화) 09:00[2024년 금융위원회 정책 돋보기 ④]오늘부터 모든 주택의 전세대출을쉽고 편리하게 갈아탈 수 있습니다.- 「온라인·원스톱 대환대출 인프라」 대출 갈아타기 서비스,신용대출·주택담보대출에 이어 전세대출까지 확대  금융당국은 금융의 디지털 전환과 금융권 경쟁 촉진을 통해 국민들의 대출 이자 부담이 경감될 수 있도록 ‘23.5.31일 「온라인·원스톱 대환대출 인프라(이하 ‘대환대출 인프라’)」를 구축하였다. 대환대출 인프라를 통해 


[메타데이터]

{'producer': 'Call PDF v 2.4', 'creator': 'Call PDF', 'creationdate': '', 'title': '', 'author': '', 'subject': '', 'source': '..\\data\\조간보도자료.pdf', 'total_pages': 10, 'page': 3, 'page_label': '4'}
========= [앞부분] 미리보기 =========
- 4 -
  전세대출 갈아타기 시 주의할 점은, 기존 대출의 대출보증*을 제공한 보증기관과 동일한 보증기관의 보증부 대출로만 갈아타기가 가능하다는 점이다. 예를 들어 한국주택금융공사(HF) 보증부 대출을 받은 차주의 경우, 대출 갈아타기는 한국주택금융공사의 보증부 대출상품으로만 가능하다. 이는 보증기관별로 대출보증 가입요건, 보증 한도, 반환보증 가입 의무 등이 상이하

In [3]:
# 2. 불러온 pdf를 청크로 나누고, 인덱싱
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)

splits = text_splitter.split_documents(docs)
len(splits)

26

In [4]:
# 3. 벡터스토어를 생성
vectorstore = FAISS.from_documents(documents=splits, embedding=OpenAIEmbeddings(model="text-embedding-3-small"))

In [5]:
# 4. 뉴스에 포함되어 있는 정보를 검색하고 생성
retriever = vectorstore.as_retriever()

prompt = PromptTemplate.from_template(
    """당신은 질문-답변(Question-Answering)을 수행하는 친절한 AI 어시스턴트입니다. 당신의 임무는 주어진 문맥(context) 에서 주어진 질문(question) 에 답하는 것입니다.
검색된 다음 문맥(context) 을 사용하여 질문(question) 에 답하세요. 만약, 주어진 문맥(context) 에서 답을 찾을 수 없다면, 답을 모른다면 `주어진 정보에서 질문에 대한 정보를 찾을 수 없습니다` 라고 답하세요.
한글로 답변해 주세요. 단, 기술적인 용어나 이름은 번역하지 않고 그대로 사용해 주세요.

#Question:
{question}

#Context:
{context}

#Answer:"""
)

llm = ChatOpenAI(model_name="gpt-5-nano", temperature=0)


In [6]:
# 5. 체인을 생성
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [7]:
answer = rag_chain.invoke("전세 대출 서비스에 대해서 문의 할 수 있는 연락처 정보 알려줘.")
print(answer)

주어진 정보에서 질문에 대한 정보를 찾을 수 없습니다


In [8]:
answer = rag_chain.invoke("금융위원회에서 발간한 서비스 요약해줘.")
print(answer)

다음은 금융위원회가 발간한 서비스 요약입니다.

- 서비스명: 온라인·원스톱 대환대출 인프라(대환대출 인프라)
- 적용 범위: 신용대출 갈아타기, 주택담보대출 갈아타기, 전세대출 갈아타기
- 시작 시점
  - 신용대출 갈아타기: 운영 중 (23.5.31부터)
  - 주택담보대출 갈아타기: 운영 시작(24.1.9)
  - 전세대출 갈아타기: 개시(24.1.31)
- 대상 확대 계획
  - 실시간 시세 조회 대상 확대: 빌라, 오피스텔 등까지 24.6월 말까지 확대 예정
- 전세대출 관련 검토 사항
  - 전세대출 보증기관의 보증기준 등으로 인한 대출 갈아타기 가능 기간이 현재 제한되어 있는데, 이를 확대하는 방안을 보증기관 등과 협의하여 검토 중
- 향후 방향
  - 대출 갈아타기 서비스 이용 현황을 면밀히 모니터링하고, 보다 많은 금융소비자가 편리하게 이용할 수 있도록 서비스 개선방안을 지속 검토

요약하면, 온라인·원스톱 대환대출 인프라를 통해 신용대출, 주택담보대출, 전세대출 갈아타기를 제공하고, 대상 범위와 이용 편의성 확장을 지속 추진한다는 내용입니다.
